In [1]:
from gurobipy import *
import numpy as np
import csv
import os
import pandas as pd
import math

In [2]:
file1 = "ZipInfoCleaned.csv"
file2 = "POD SitesNew.csv"
#file3 = "Allegheny_population.csv"
file3 = "HospitalSites.csv"
file4 = "Scenarios.csv"

In [3]:
zipcode = pd.read_csv(file1,delimiter=",")
podsites = pd.read_csv(file2,delimiter=",")
#population = pd.read_csv(file3,delimiter=",")
hospitals = pd.read_csv(file3,delimiter=",")
scenarios = pd.read_csv(file4,delimiter=",")

In [4]:
zipcode = zipcode[['ZIP', 'LAT', 'LON', 'Population']].dropna()
#zipcode_np = zipcode[['LAT', 'LON']].to_numpy()
#zipcode_np
#merged_zipcode = population.merge(zipcode, left_on = "ZIP", right_on = "ZIP")
zipcode_np = zipcode[['LAT', 'LON', 'Population']].to_numpy()

In [5]:
podsites1 = podsites[["SCHOOL/FACILITY NAME", "Latitude", "Longitude"]].dropna()
podsites_np = podsites1[["Latitude","Longitude"]].to_numpy()

HospitalSites = hospitals[["Address for the hospital", "Latitude_hospital", "Longitude_hospital"]].dropna()
HospitalSites_np = HospitalSites[["Latitude_hospital","Longitude_hospital"]].to_numpy()

In [6]:
scenarios_merged = scenarios.merge(zipcode, left_on = "ZIP", right_on = "ZIP")
scenarios_merged = scenarios_merged.drop(["LAT","LON","Population"], axis=1)
scenarios_np = scenarios_merged.to_numpy()

In [7]:
#calculate empty list of lists rows = podsites, cols = zips
distance_zip = [[0 for i in range(len(zipcode_np))] for j in range(len(podsites_np))]
np.shape(distance_zip)

(47, 92)

In [8]:
#calculate empty list of lists rows =podsites, cols = hospitals
distance_hos = [[0 for i in range(len(HospitalSites_np))] for j in range(len(podsites_np))]
np.shape(distance_hos)

(47, 14)

In [9]:
# Indices for POD sites and zipcodes
i = len(podsites_np) # Potential POD sites
j = len(zipcode_np) # All the zipcodes
sites = range(i) # Equivalently, sites = [0,1,2,3,4]
zips= range(j) # Equivalently, zips = [0,1,2,3]
scenarios = scenarios_np
scenarios = range(9)

# calculate haversine distance
for i in sites:
    for j in zips:
        #distance between sites (cols) and zips (rows)
        
        #Haversine Distance
        #earth radius in meters
        R = 6373000.

        lat1 = np.radians(podsites_np[i,0])
        lon1 = np.radians(podsites_np[i,1])
        lat2 = np.radians(zipcode_np[j,0])
        lon2 = np.radians(zipcode_np[j,1])

        phi1 = np.radians(lat1)
        phi2 = np.radians(lat2)
        delta_phi = np.radians(lat2 - lat1)
        delta_lambda = np.radians(lon2 - lon1)
        a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
        res = R * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
        distance_zip[i][j] = np.round(res, 2)

In [10]:
# Indices for POD sites and Hospitals
i = len(podsites_np) # Potential POD sites
j = len(HospitalSites_np) # All the zipcodes
sites = range(i) # Equivalently, sites = [0,1,2,3,4]
hospitals= range(j) # Equivalently, hospital = [0,1,2,3]

# calculate haversine distance
for i in sites:
    for j in hospitals:
        #distance between sites (cols) and zips (rows)
        
        #Haversine Distance
        #earth radius in meters
        R = 6373000.

        lat1 = np.radians(podsites_np[i,0])
        lon1 = np.radians(podsites_np[i,1])
        lat2 = np.radians(HospitalSites_np[j,0])
        lon2 = np.radians(HospitalSites_np[j,1])

        phi1 = np.radians(lat1)
        phi2 = np.radians(lat2)
        delta_phi = np.radians(lat2 - lat1)
        delta_lambda = np.radians(lon2 - lon1)
        a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
        res = R * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
        distance_hos[i][j] = np.round(res, 2)

In [11]:
DIST1 = np.array(distance_zip)
DIST2 = np.array(distance_hos)

In [12]:
D = scenarios_np

In [13]:
P = [[1/9],[1/9],[1/9],[1/9],[1/9],[1/9],[1/9],[1/9],[1/9]]

In [14]:
wage = 14.37
cost1 = wage *8*260/20
B = 0

In [15]:
D[1]

array([15017.  ,  4053.25,     0.  ,     0.  ,     0.  ,     0.  ,
       16213.  ,  4053.25,     0.  , 12159.75])

In [16]:
# Setting up model object
m = Model("POD")

Using license file c:\gurobi903\gurobi.lic
Academic license - for non-commercial use only


In [17]:
# sites/zips/hospitals/scenarios

In [18]:
## Decision Variable

# 1st stage
x = m.addVars(sites, vtype=GRB.BINARY)
st = m.addVars(sites, lb=0.0, vtype=GRB.INTEGER)
z = m.addVars(hospitals,sites, vtype=GRB.BINARY)

# 2nd stage
y = m.addVars(sites,zips,scenarios,vtype = GRB.BINARY)
b = m.addVars(hospitals, sites, scenarios, lb=0.0, vtype=GRB.INTEGER)


In [19]:
## Objective Function

# Objective 1
obj1_1 = LinExpr()
obj1_2 = LinExpr()
for s in scenarios:
    for i in sites:
        for j in zips:
            obj1_1 += DIST1[i,j]*D[j,s]*y[i,j,s]*1/9

for s in scenarios:
    for i in sites:
        for h in hospitals:
            obj1_2 += DIST2[i,h] * b[h,i,s] *1/9
m.setObjective(obj1_1+obj1_2, GRB.MINIMIZE)

In [20]:
M = 100000

In [21]:
## Constrains

# Demand constraint: The total resources at a POD (initial stock+surge) should meet the demand of all neighborhoods that are assigned to it.
for s in scenarios:
    for i in sites:
        m.addConstr((sum(b[h,i,s] for h in hospitals) + st[i]) >= sum(D[j,s] * y[i,j,s] for j in zips))
        
# Budget constraint
m.addConstr(sum(st[i]*cost1 for i in sites) <= 0.4*B)

            
# One neighborhood (per zip code) can have (at most?) only one POD       
for j in zips:
    for s in scenarios:
        m.addConstr(sum(y[i,j,s] for i in sites) == 1) ### DISCUSS <=/==
        
# Neighborhoods cannot be assigned to a POD that is not constructed.
for i in sites:
    for j in zips:
        for s in scenarios:
            m.addConstr(y[i,j,s] <= x[i]) 

# Only if a POD is built and it is assigned to a hospital, supplies can be transported from that hospital to that POD
for i in sites:
    for h in hospitals:
        for s in scenarios:
            m.addConstr(b[h,i,s] <= M*
                        x[i]) 
for i in sites:
    for h in hospitals:
        for s in scenarios:
            m.addConstr(b[h,i,s] <= M*z[h,i])
            
#Only if a POD is built there can be initial storage
for i in sites:
    m.addConstr(st[i] <= M*x[i])
            
        
        
        

In [22]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 52059 rows, 45590 columns and 166239 nonzeros
Model fingerprint: 0x0669e203
Variable types: 0 continuous, 45590 integer (39621 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [2e+00, 5e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 51920 rows and 40608 columns
Presolve time: 0.28s
Presolved: 139 rows, 4982 columns, 9306 nonzeros
Variable types: 0 continuous, 4982 integer (4324 binary)
Found heuristic solution: objective 8.863512e+07

Root relaxation: objective 5.771705e+07, 217 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5.7717e+07    0   12 8.8635e+07 5.7717e+07  34.9%     -    0s
H    0     0                    5.786899e+07 5.7717e+07  0.26%     -    0s
H    0     0                    5

In [23]:
#check if it worked
totalzip_assigned = 0
total_Sites =0
for s in scenarios:
    totalzip_assigned = 0
    for i in sites:
        for j in zips:

            if y[i,j,s].x == 1:
                totalzip_assigned +=1
    print(totalzip_assigned)


for i in sites:
    if x[i].x == 1:
        total_Sites+=1

print(total_Sites)  


92
92
92
92
92
92
92
92
92
47


In [24]:
zip_index =[]
pod_index =[]
pod_zip_pair =[]
for i in sites:
    for j in zips:
        if y[i,j,2].x == 1:
            pod_index.append(i)
            zip_index.append(j)
            pod_zip_pair.append((i,j))

model_distance =[]            
for pair in pod_zip_pair:
    model_distance.append(distance_zip[pair[0]][pair[1]])

In [25]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(model_distance)
plt.title("Model distacne (in mtrs) Distribution")
plt.xlabel("distance")
plt.ylabel("Number of pairs")
plt.show()

In [ ]:
x = m.addVars(sites, vtype=GRB.BINARY)
y = m.addVars(sites,zips, vtype = GRB.BINARY)
st = m.addVar(lb=0.0, vtype=GRB.INTEGER)

z = m.addVars(hospitals,sites, vtype=GRB.BINARY)

In [ ]:
m.setObjective (sum(sum((distance[i][j]*y[i,j]*zipcode_np[j,2]) for i in sites) for j in zips))
m.modelSense = GRB.MINIMIZE

In [ ]:
#capacity constraint
for i in sites:
    m.addConstr(sum(zipcode_np[j,2]*y[i,j] for j in zips ) <= z[i]*x[i] )
    
# number of pod
m.addConstr(sum(x[i] for i in sites) <= 10)# number of pod?

#one zip area to one pod
for j in zips:
    m.addConstr(sum(y[i,j] for i in sites) == 1)
for j in zips:
    for i in sites:
        m.addConstr(y[i,j] <= x[i])   

In [ ]:
m.optimize()

In [ ]:
zip_index =[]
pod_index =[]
pod_zip_pair =[]
for i in sites:
    for j in zips:
        if y[i,j].x == 1:
            pod_index.append(i)
            zip_index.append(j)
            pod_zip_pair.append((i,j))

model_distance =[]            
for pair in pod_zip_pair:
    model_distance.append(distance[pair[0]][pair[1]])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
model_distance


In [ ]:
plt.hist(model_distance)
plt.title("Model distacne (in mtrs) Distribution")
plt.xlabel("distance")
plt.ylabel("Numbe of pairs")
plt.show()